# Setup

This Jupyter notebook uses the necessary Python packages to connect to a Redshift database and execute queries in .SQL files in the 'queries' directory. Each question in the assessment is listed with a print_query call to show the script to be run against the db. I present in this format as it would be easy for a colleague to  add their own sql files to the directory and quickly begin analyses without having to change much of the notebook.

To run the queries and view the results requires some setup steps:
- It is recommended to use a virtual environment, like venv
- In your terminal, change working directory to the root of this repo and run 'pip install -r requirements.txt' to install dependencies
- Execute the IMPORT block below
- Execute the 3 METHODS blocks below; this includes the start_engine method to create the connection to the Redshift db

## Import

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# SQLAlchemy is the ideal package for connecting to redshift RDMS as it is better supported by pandas; read, doesn't throw the 'unsupported' warning
import sqlalchemy as sa
from sqlalchemy.engine.url import URL
from sqlalchemy import orm as sa_orm

## Methods

In [ ]:
# build the sqlalchemy URL
def start_engine():
    url = URL.create(
    drivername='redshift+redshift_connector', # indicate redshift_connector driver and dialect will be used
    host='', # Amazon Redshift host
    port=5439, # Amazon Redshift port
    database='', # Amazon Redshift database
    username='', # Amazon Redshift username
    password='' # Amazon Redshift password
    )
    engine = sa.create_engine(url)
    Session = sa_orm.sessionmaker()
    Session.configure(bind=engine)
    return engine

In [ ]:
def print_query(file_path):
    f = open(file_path, 'r')
    file_contents = f.read()
    print (file_contents)

In [ ]:
#
def execute_query(file_path):
    engine = sa.create_engine(url)
    Session = sa_orm.sessionmaker()
    Session.configure(bind=engine)

    # Read SQL code from .sql file
    with open(file_path, "r") as file:
        sql = file.read()

    # Execute SQL code and load results into a pandas dataframe
    df = pd.read_sql(sql, engine)

    # Close connection
    engine.dispose()
    return df

# Questions

### *Which are the top 10 account ids by number of users?*

In [ ]:
print_query('queries/top10.sql')

In [ ]:
execute_query('queries/top10.sql')

### *Create a summary table at the account level that signals when an account is new (boolean). An account is new for the first day we see it run a task(s).*

In [ ]:
print_query('queries/first_account.sql')

In [ ]:
create_table_new_accounts=execute_query('queries/new_accounts.sql')
create_table_new_accounts

### *Add another column with the moving average of the tasks run in the last 7 days for each account.*

In [ ]:
print_query('queries/seven_day_avg.sql')

In [ ]:
seven_day_avg=execute_query('queries/seven_day_avg.sql')
seven_day_avg

### *Create a summary table at the account level. Add a column with the % difference in the number of tasks to the previous day*

In [ ]:
print_query('queries/seven_day_avg_and_daily_pct_chg.sql')

In [ ]:
start_engine()
pct_change=execute_query('queries/seven_day_avg_and_daily_pct_chg.sql')
pct_change

### *A lost account is an account with no tasks run on a given month. How many accounts did we lose (had no executed tasks) in February 2017?*

In [ ]:
print_query('queries/lost_accts.sql')

In [ ]:
start_engine()
lost_accounts=execute_query('queries/lost_accts.sql')
lost_accounts

### *(OPTIONAL) Create a visualization that represents the growth of new accounts in a way you would communicate to a peer or business stakeholder*

In [ ]:
start_engine()

# Fetch data for new accounts


query = "SELECT date, count(account_id) as new_accounts FROM first_account_tasks WHERE is_new = 't' GROUP BY date ORDER BY date"
new_accounts = pd.read_sql(query, engine)

# Plot line chart of new accounts growth
plt.plot(new_accounts['date'], new_accounts['new_accounts'], label='New Accounts')
plt.xlabel('Date')
plt.ylabel('Number of New Accounts')
plt.title('New Accounts Over Time')
plt.legend()
plt.show()
